# TOÁN ỨNG DỤNG VÀ THỐNG KÊ

## Đồ án cuối kì

---

**Thông Tin Sinh Viên**
- Lớp: CQ2022/3
- Họ và tên: Trương Tiến Anh
- MSSV: 22120017

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import linear_model

# **Xây dựng mô hình**

### **Linear Regression**

In [9]:
# Linear regression
def linear_regression(features, target, fit_intercept=True):
    try:
        model = linear_model.LinearRegression(fit_intercept=fit_intercept).fit(features, target)
        return model.coef_, model.intercept_
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

### Ý tưởng
##### Giới thiệu

* **Xây dựng mô hình đơn giản theo công thức: $Ax = b$**
  * $A$ là ma trận dữ liệu (các tính chất của rượu)
  * $b$ là nhãn (label) của mỗi dòng dữ liệu trong ma trận $A$, được thể hiện dưới dạng đánh giá thống kê

##### Công thức mô hình

* **Với mô hình trên, ta có thể tìm mô hình theo công thức:**
  $\hat{x}=A^\dagger\cdot b$
  * Khi đó, mô hình sẽ đi qua gốc tọa độ của đồ thị, điều này dẫn đến sự hạn chế.
  * Do đó, chúng ta chọn mô hình theo công thức sau:
  \[Ax + b_0 = b\]
  * Với công thức này, mô hình có thể dịch chuyển trên trục tung, tăng sự linh hoạt.

##### Sử dụng `sklearn` để xây dựng mô hình

* Bộ thư viện `sklearn` cung cấp công cụ để dựng mô hình hồi quy tuyến tính với công thức đã chọn:
  * Sau khi fit các tham số bao gồm \(A\) và \(b\) vào mô hình, các thuộc tính quan trọng của mô hình Linear Regression được sinh ra, bao gồm:
    * `coef_`: \(\hat{x}\)
    * `intercept_`: \(b_0\)
  * Gán các giá trị này vào công thức, ta được mô hình hồi quy tuyến tính cần tìm.



### **Cross Validation**

In [10]:
# Cross Validation
def cross_validation(features, target, k_folds):
    kf = KFold(n_splits=k_folds)
    error_list = []

    for train_indices, test_indices in kf.split(features):
        features_train, features_test = features[train_indices], features[test_indices]
        target_train, target_test = target[train_indices], target[test_indices]
        coefficients, intercept = linear_regression(features_train, target_train)
        
        if coefficients is None or intercept is None:
            print("An error occurred during model training.")
            return None
        
        predictions = np.dot(features_test, coefficients) + intercept
        error = np.mean(np.abs(target_test - predictions))
        error_list.append(error)

    return np.mean(error_list)

### Ý tưởng
##### Giới thiệu
* **K-Fold Cross-validation** là một kỹ thuật dùng để đánh giá hiệu quả của mô hình trên tập dữ liệu.
  * **Xáo trộn dữ liệu** (tùy chọn)
  * **Chia dataset thành k nhóm** (folds)
  * **Quy trình thực hiện với mỗi nhóm**:
    1. Sử dụng nhóm hiện tại để đánh giá hiệu quả mô hình.
    2. Các nhóm còn lại được sử dụng để huấn luyện mô hình.
    3. Huấn luyện mô hình trên tập train.
    4. Đánh giá mô hình trên tập test.
  * **Tổng hợp hiệu quả qua các đánh giá**.
##### Sử dụng `sklearn` cho K-Fold Cross-validation
* Bộ thư viện `sklearn` cung cấp hàm `sklearn.model_selection.KFold` để tự động chia tập dữ liệu thành \(k\) nhóm và `split` thành các bộ dữ liệu train/test khác nhau.
  * **Quy trình chi tiết**:
    1. **Xây dựng mô hình** trên tập train, ta được các tham số $\hat{x}$, $b_0$
    2. **Áp dụng mô hình lên tập test**: $A_{test}\cdot \hat{x}=b'$
    3. **Tính sai số** so với nhãn của tập test: $|b'-b_{test}|$. Kết quả này là một ma trận có shape giống  $b_{test}$. Tính trung bình của ma trận này để được sai số của mô hình trên tập train/test đó.
    4. **Lặp lại cho tất cả các tập train/test** được split ra ở trên, sau đó tính trung bình các sai số này để được sai số trung bình của mô hình dựa trên phương pháp Cross Validation.
  * **Số \(k\) được chọn cho K-Fold thường là 10**.

### **Đọc File CSV**
* Tách dữ liệu đọc từ `NHANES_age_prediction.csv` thành bộ dữ liệu và bộ label

In [7]:

# Đọc file CSV
df = pd.read_csv('NHANES_age_prediction.csv')

# Bỏ qua các cột "SEQN" và "age_group"
df = df.drop(columns=['SEQN', 'age_group'])

# Chuẩn bị dữ liệu
features = df.drop(columns=['DIQ010']).to_numpy()  # Sử dụng tất cả các cột ngoại trừ cột "DIQ010"
target = df['DIQ010'].to_numpy()                   # Sử dụng cột "DIQ010" làm nhãn
labels = df.drop(columns=['DIQ010']).columns       # Lấy tên các cột đặc trưng

print("Features (Đặc trưng):")
print(features)
print("\nTarget (Nhãn):")
print(target)
print("\nLabels (Tên các cột đặc trưng):")
print(labels)

Features (Đặc trưng):
[[ 61.     2.     2.   ... 110.   150.    14.91]
 [ 26.     2.     2.   ...  89.    80.     3.85]
 [ 16.     1.     2.   ...  89.    68.     6.14]
 ...
 [ 34.     1.     2.   ... 103.   124.    21.41]
 [ 60.     2.     2.   ...  90.   108.     4.99]
 [ 26.     1.     2.   ... 108.   108.     3.76]]

Target (Nhãn):
[2. 2. 2. ... 2. 2. 2.]

Labels (Tên các cột đặc trưng):
Index(['RIDAGEYR', 'RIAGENDR', 'PAQ605', 'BMXBMI', 'LBXGLU', 'LBXGLT',
       'LBXIN'],
      dtype='object')


# **Xây dựng mô hình dự báo bệnh tiểu đường (biến "DIQ010") bằng phương pháp hồi quy tuyến tính**

### **Câu a. Sử dụng bộ 7 đặc trưng đề bài cung cấp.**

In [11]:

coefficients, intercept = linear_regression(features, target)
cv_error = cross_validation(features, target, k_folds=10)

# In kết quả
coefficients_str = "\n".join([f"Feature {i+1}: {coef:.6f}" for i, coef in enumerate(coefficients)])
print("Model Coefficients:\n" + coefficients_str)
print(f"Intercept: {intercept:.6f}")
print(f"Cross-Validation Error (10-fold): {cv_error:.6f}")

Model Coefficients:
Feature 1: 0.000583
Feature 2: -0.014352
Feature 3: 0.000453
Feature 4: 0.000315
Feature 5: -0.000440
Feature 6: 0.000011
Feature 7: 0.001255
Intercept: 2.031667
Cross-Validation Error (10-fold): 0.052440


### Ý tưởng
* Dùng `linear_regression()` đã cài đặt để tìm model trên tất cả các tính chất

### **Câu b. Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất. (Dùng phương pháp Cross Validation)**

In [12]:
# Choose best property
def choose_best_property(X, y):
    cross_val_errors = []

    for feature_index in range(X.shape[1]):
        cross_val_error = cross_validation(X[:, feature_index:feature_index + 1], y, k_folds=10)
        if cross_val_error is not None:
            cross_val_errors.append(cross_val_error)
        else:
            cross_val_errors.append(np.inf)  # Nếu có lỗi, gán giá trị lỗi rất lớn để không chọn đặc trưng này
    
    best_property_index = np.argmin(cross_val_errors)
    return cross_val_errors, best_property_index

In [13]:
error_list, best_index = choose_best_property(features, target)

# Xây dựng mô hình với tính chất tốt nhất
best_feature = features[:, best_index:best_index + 1]
coefficients, intercept = linear_regression(best_feature, target)

# In kết quả
print("\n=== Best Property Selection ===")
print(f"Best property: {labels[best_index]}")
print("\n=== Model ===")
print(f"Model: y = {coefficients[0]:.6f} * x + {intercept:.6f}")
print("\n=== Cross-Validation Error ===")
print(f"Cross-Validation Error: {error_list[best_index]:.6f}\n")


=== Best Property Selection ===
Best property: RIDAGEYR

=== Model ===
Model: y = 0.000460 * x + 1.997015

=== Cross-Validation Error ===
Cross-Validation Error: 0.049779



### Ý tưởng
* Đối với từng cột (tính chất), chạy hàm `cross_validation()` để tìm sai số của mô hình dựa trên mỗi tính chất.

* Tìm tính chất có sai số bé nhất để xác định tính chất tốt nhất.

* Chạy `linear_regression()` để tìm mô hình dựa trên tính chất tốt nhất này.

### **Câu c. Xây dựng một mô hình của riêng bạn cho kết quả tốt nhất**

In [14]:
# Build model
def build_model(X, y, error_list, max_features):
    sorted_error_indices = np.argsort(error_list)
    full_error = cross_validation(X, y, k_folds=10)

    abs_error_list = []
    cv_error_list = []
    selected_features_list = []

    for num_features in range(2, max_features - 1):
        selected_features = sorted_error_indices[:num_features]
        X_selected = X[:, selected_features]
        cv_error = cross_validation(X_selected, y, k_folds=10)
        
        if cv_error is not None:
            cv_error_list.append(cv_error)
            abs_error_list.append(np.abs(full_error - cv_error))
            selected_features_list.append(selected_features)
        else:
            cv_error_list.append(np.inf)
            abs_error_list.append(np.inf)
            selected_features_list.append(selected_features)
    
    best_model_index = np.argmin(abs_error_list)
    return cv_error_list, selected_features_list, best_model_index

In [15]:
cv_error_list, prop_list, best_index_prop = build_model(features, target, error_list, len(labels))

# Xây dựng mô hình với top x tính chất tốt nhất
best_properties = features[:, prop_list[best_index_prop]]
coefficients, intercept = linear_regression(best_properties, target)

# In kết quả
print("\n=== Model Building with Top Features ===")
print("Best properties:")
for i, prop in enumerate(labels[prop_list[best_index_prop]], start=1):
    print(f"{i}. {prop}")

print("\nModel:")
print(f"A{coefficients} + {intercept} = b")

print("\nCross-Validation Error:", cv_error_list[best_index_prop])


=== Model Building with Top Features ===
Best properties:
1. RIDAGEYR
2. LBXIN
3. BMXBMI
4. PAQ605
5. RIAGENDR

Model:
A[ 0.00049905  0.00111593  0.00022985  0.00035016 -0.01218377] + 1.993528452915252 = b

Cross-Validation Error: 0.051283847762980804


### Ý tưởng
* Chọn ra n tính chất tốt nhất (cho n chạy từ 2 đến 10) rồi chạy `cross_validation()`

* Chọn ra bộ tính chất tốt nhất (sai số thấp nhất), các tính chất sẽ được chọn để dựng mô hình

* Chạy `linear_regression()` để tìm model dựa trên các tính chất này

* Kết quả của mô hình tự xây dựng và của mô hình 7 tính chất:

    * Mô hình tự xây dựng: 0.051284
    * Mô hình 7 tính chất: 0.052440